In [45]:
# Import dependencies
import pandas as pd
import pymongo
from pycorenlp import StanfordCoreNLP

## Connect to database

In [46]:
# Create connection variable
conn = 'mongodb://localhost:27017'

# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)

# Connect to a database. Will create one if not already available.
db = client.tweets

## Retrieve a tweet from database

In [78]:
# search tweets in database using relevant keywords
query = { "text": { "$regex": '.*market.*|.*SPX.*|.*stock.*|.*fed.*|.*econom.*|.*bull.*|.*bear.*|.*momentum.*|.*volat.|.*treasury.*|.*powell.*|.*policy.*|.*spending.*|.*tariff.*|.*trade.*|.*SPY.*|.*tax.*|.*interest.', 
                   "$options": 'i'}} 


tweets = db['capstone_finance'].find(query)

# build a dataframe 
df = pd.DataFrame(list(tweets))

In [80]:
# Create a new column for sentiment
df['sentiment_by_sentence'] = ''

## Sentiment analysis with CoreNLP

In [102]:
def analyze_sentiment(tweet):
    
    # The functional analyses a given text (a string) and returns a sentiment of every senence as a list of strings
    
    sentimens = []
    
    try:
    
        result = nlp.annotate(tweet,
                              properties={
                                  'annotators': 'sentiment, ner, pos',
                                  'outputFormat': 'json',
                                  'timeout': 1000,
                              })

        sentiments = [i["sentiment"] for i in result["sentences"]]
        
    except:
        print("Error")
        return "nan"
    
    return sentiments
                       

In [103]:
# Apply analyze sentiment function to eery tweet in the DataFrame
df['sentiment_by_sentence'] = df['text'].apply(analyze_sentiment)

A trainee day trader in France made $11.6 million in one day on what he thought was just a “demo.” https://t.co/kU2MwWHUf5
Krispy Kreme opening huge store in Times Square—and it will have a 'doughnut-glaze waterfall' (via @CNBCMakeIt) https://t.co/MV1NdJQGiC
Trump says 'there will be no reduction in the tariffs' imposed against China https://t.co/OUP5Tr7Vwn
'It's a temporary time out': Trump and Xi agree to talks, but offer no clear path to end trade war https://t.co/LFjfiypUtD
The Trump economy is starting to look more and more like the Obama economy. https://t.co/xn91LcfYpH https://t.co/KleE0eEto3
US business groups praise Trump's trade war truce, but want an 'enforceable agreement' with China https://t.co/6HxLRfqpNF
This buyback-focused ETF is beating the market—here's how $TTAC (via @ETFEdgeCNBC) https://t.co/zqKNhverHz
In this #CNBCClassic clip from 1995, the Dow breaks 5,000 for the first time as traders react on the floor of the @NYSE. The Dow is now above 26,000. https://t.co/T

RT @puiwingtam: Don't pay through the nose for cell service if you're traveling internationally this summer. @bxchen has more economical so…
Bragging rights for the world’s hottest major stock market this year belong to an unlikely leader: President Putin of Russia https://t.co/STSnVzA5cC
RT @tiffkhsu: Franchisees say they are being pushed out through a “rigged” compliance process where, due to a conflict of interest baked in…
A majority of voters think Mr. Trump's trade policies will hurt the American economy, and even Republicans think they will raise prices, a survey for The Times finds https://t.co/a7Ya6oAsWf
The Federal Aviation Administration said it had discovered a new problem with the 737 Max jet that Boeing must correct before the plane is returned to service https://t.co/NnrGG8921N
The Federal Reserve said the largest banks were generally well prepared to withstand a severe crisis, although Credit Suisse must improve its operations https://t.co/WL28KbCEnP
StockX, a website t

RT @TheRealLSL: IPO Buyers Think Big With Addressable Markets - for ⁦@WSJheard⁩  https://t.co/oF9i8jpYnI
Yields on Treasury inflation-protected securities have declined to near their lowest levels in almost two years. https://t.co/sOEDbTrSWf
RT @CocoF1026: China is insisting that the U.S. remove its Huawei ban. Beijing also wants the U.S. to lift all punitive tariffs and drop ef…
RT @srussolillo: Hong Kong’s an­ti­cor­rup­tion watch­dog ar­rested a for­mer high-rank­ing of­fi­cial at the city’s stock ex­change for al…
RT @nate_taplin: Heard on the Street: A new round of tariffs would severely knock both U.S. and Chinese growth—but isn’t inevitable  https:…
What do Jane Street Capital, the United Church of Christ and San Bernardino County have in common? They are all fighting an SEC proposal they say will skew bond markets in favor of large Wall Street firms. https://t.co/rdmzuHezJ8
European stocks follow Asian indexes higher https://t.co/NpSVYIluVb
RT @Birdyword: Trump has one very pow

Content, curated: See analysis, articles and perspectives from St. Louis Fed contributors on Medium https://t.co/nviX9qNrnu
On June 26, #GDPNow model from @AtlantaFed was projecting real GDP growth in Q2 of 1.9%, down from 2.0% a week earlier https://t.co/q47kux7G0B https://t.co/FXdtwDO6kl
St. Louis Fed Financial Stress Index edges lower for a second week, dipping to -1.246 from -1.217 a week earlier (normal stress=0) https://t.co/836vqiJyWF https://t.co/USaP6OctTF
From the newest #WomeninEconomics podcast: Hear @KansasCityFed President Esther George discuss growing up in rural Missouri and working in a male-dominated field https://t.co/8yZDCyZi5Y https://t.co/tGRwX7i9lC
In a Bloomberg interview, Bullard discussed his dissent at the FOMC’s June meeting: “It seemed to me like this was a good chance to make an insurance rate cut and try to re-center inflation and inflation expectations back at the 2% target” https://t.co/k3vDJ07qrW
In a two-part post, the FRED Blog is explaining recent c

Register now for the Fed’s #MinoritiesInBanking Forum Sept. 26-27 in St. Louis. Early bird pricing is $200 through June 30 https://t.co/Tlri8GDv2H https://t.co/CAOqxn91ga
Why are the Mississippi and Indiana state seals embedded in the sidewalk outside the St. Louis Fed? https://t.co/mVAEa9RR5e https://t.co/EGoJoKdGvU
How do examiners use both quantitative and qualitative inputs to evaluate a bank’s performance? #STLFedCD https://t.co/EWOrYvxcOk https://t.co/0hswMaDGRr
RT @TheStreet: Facebook's Libra: @EricJhonsa Says Emerging Markets and 'Microtransactions' Might Hold the Key  https://t.co/VBF1SkGGw4
RT @TheStreet: For more on stock 'orphans' and what they mean for the market, read @JimCramer's full take on Slack on @RealMoney: https://t…
RT @TheStreet: .@JimCramer breaks down Canopy Growth's mission to be the only winner in the legal cannabis space. More on #StockoftheDay $C…
RT @TheStreet: .@Sarge986 has one word on #StockoftheDay $CGC and it isn't pretty https://t.co/A0jZwGUxm4
RT @

...

Error
Sir John Templeton: “Bull markets are born on despair, grow on skepticism, mature on optimism and die on euphoria.” https://t.co/4ELPZysDiY
RT @SchwabResearch: Schwab Market Update: Stocks moving higher as employment and services sector data appear to be keeping Fed expectations…
Keep this in mind: Not to rain on the #Apple1T parade, but FWIW, owning the highest-cap stock in the S&amp;P 500 has returned 972% since 1972 versus a 10,214% return from the index. @NDR_Research
Services #ISM fell more than expected from 59.1 in June to 55.7 in July: employment up, but new orders, business activity &amp; supplier deliveries all fell...prices paid up to 63.4 from 60.7 (#inflation)...#TradeWar &amp; #tariffs having impact as per respondents’ comments
#TradeDeficit jumped to $46.3b in June from $43.2b in May: real goods imports +0.8% m/m vs. real goods exports -1.4% m/m
RT @KathyJones: #BOE raises rates and issues a hawkish statement but GBP is down. Brexit and trade wars are more importan

@Nicholas_Boyer Full-on trade war would be more damaging IMO as it would have global implications &amp; risk stagflation ... inverted yield curve still a bit in distance &amp; as my other post showed this morning; from current spreads, doom has not descended historically
#TradeWar: further escalation = largest risk for market/economy &amp; today’s immense/integrated/complicated global supply chains ill-equipped to withstand tariffs’ pressures = higher inflation/lower growth (here &amp; globally)
% stocks trading &gt; 200-DMA (blue line) diverging from S&amp;P (white line) lately https://t.co/YPdl0TmdIN
Detailed historical look at #Recession timing &amp; stock market behavior relative to #YieldCurve #Inversions (both 10y-2y &amp; 10y-3m) from around current spreads https://t.co/2W9IkIEeB6
Top 5 stocks have often had higher weight than bottom half...since @NDR_Research common stock data set began in 1972, avg weight of top 5 stocks has been 14.3% vs 11.7% for bottom half (bottom half’s c

RT @SchwabResearch: The Alternative Minimum Tax exemption for tax year 2018 for married taxpayers filing jointly, up from $84,500 in 2017.…
Just to clarify @cnbc it’s 10% tariffs on $200b worth of goods; not $200b in China tariffs ... I’m no fan of the #tradewar, but headlines should be accurate https://t.co/gdZpSkhnL2
#TradeWar taking its toll on #capex plans &amp; business optimism
@MarcumLLP @GoldmanSachs @SoberLook https://t.co/zVqS23ylbD
S&amp;P hit 3m high yesterday for 1st time since Jan but internals less robust (only 15% of stocks did same, &lt; important 30% threshold, ie performance remains bifurcated)...more than 5m since 30% of the S&amp;P met that criteria...historically, shot clock ran out at ~8m
@StrategasRP
What are the most common investor biases? #ConfirmationBias wins!
#MarketEthos @RichardsonGMP @SoberLook https://t.co/UjR9YUMuES
@DiMartinoBooth 
Post-GFC (2010), sustained final demand rebound heralded prolonged period of restocking (inventory build)...manifested i

May’s merchandise #TradeDeficit fell to $64.8b, &lt; $69b consensus w/ wholesale inventories +0.5%, retail inventories +0.4%, non-auto retail inventories +0.1%, big jump in nominal exports +2.1% &amp; imports +0.2%...should lift Q2 GDP forecasts a bit from already-high level
Looking forward to speaking shortly on my “Global Economy &amp; Corporate Strategy” panel at @StanfordLaw Directors’ College ... what a beautiful campus and day in #SiliconValley CA https://t.co/Ktgygh9hq0
RT @DianeSwonk: @HorizonGreg underscores how infighting on trade policy can confuse markets. It makes forecasting and interpreting policies…
RT @SchwabResearch: Schwab Market Update: Stocks begin week sharply lower on ramped-up trade anxiety, despite upbeat regional manufacturing…
How much does your state rely on real estate taxes? NH &amp; AK at top; ND &amp; AL at bottom
@KELoughead @taxfoundation @SoberLook https://t.co/rtDZFktqYa
U.S. stands alone among developed economies in terms of improving economic growt

Error
#FOMC does not expect 2018 economic strength to persist, projecting slower growth for 2019 &amp; 2020 (as fiscal stimulus 'sugar-high' wears off)...expects long-run rate to drop below 2% again @federalreserve @SoberLook https://t.co/vLFTktgfVA
#FOMC upped GDP &amp; inflation projections &amp; lowered forecast for unemployment rate...however, longer-run unemployment projection remains above current level
@federalreserve https://t.co/oZrqg1NWYx
One thing about the #FOMC doing pressers after every meeting: #JeromePowell is very good at them
RT @KathyJones: Best comment of the #Fed press conference. Powell, "The last two cycles didn't end with inflation. They ended with financia…
Love Jerome Powell starting presser with “I want to do a plain English summary of where we are in the economy”
#Fed hikes 25 bps to 1.75-2.00% range- highest since 10/08 in unanimous vote ... median of dots plot moved up to 2.375%, which means an expectation of 4 total hikes this year (3 expected next year)


In [105]:
# Drop the rows where sentiment_by_sentence is a nan

df_clean = df[df['sentiment_by_sentence'] != 'nan']
df_clean.head()

,_id,datetime,favorite_count,follower_count,retweet_count,screen_name,text,tweet_id,unix_timestamp,username,sentiment_by_sentence
0,5d1845236890522b32e94b6f,29-Jun-2019 23:07:14.000000,304,3192272,117,CNBC,A trainee day trader in France made $11.6 mill...,1145106478694588416,1.561875e+09,CNBC,"[Neutral, Neutral]"
1,5d1845236890522b32e94b7d,29-Jun-2019 21:55:38.000000,206,3192272,86,CNBC,Krispy Kreme opening huge store in Times Squar...,1145088459083845633,1.561871e+09,CNBC,[Negative]
2,5d1845236890522b32e94b7f,29-Jun-2019 21:44:47.000000,34,3192272,18,CNBC,Trump says 'there will be no reduction in the ...,1145085729577545728,1.561870e+09,CNBC,[Negative]
3,5d1845236890522b32e94b99,29-Jun-2019 19:59:40.000000,23,3192272,13,CNBC,'It's a temporary time out': Trump and Xi agre...,1145059277188546560,1.561864e+09,CNBC,[Negative]
4,5d1845236890522b32e94ba9,29-Jun-2019 19:04:33.000000,23,3192272,11,CNBC,The Trump economy is starting to look more and...,1145045407648686084,1.561860e+09,CNBC,"[Neutral, Neutral]"
5,5d1845236890522b32e94bd3,29-Jun-2019 15:55:53.000000,18,3192272,9,CNBC,US business groups praise Trump's trade war tr...,1144997926449455105,1.561849e+09,CNBC,[Negative]
6,5d1845236890522b32e94be1,29-Jun-2019 15:02:32.000000,12,3192272,4,CNBC,This buyback-focused ETF is beating the market...,1144984499522084865,1.561846e+09,CNBC,[Negative]
7,5d1845236890522b32e94be7,29-Jun-2019 14:41:00.000000,40,3192272,14,CNBC,"In this #CNBCClassic clip from 1995, the Dow b...",1144979081370460160,1.561844e+09,CNBC,"[Negative, Positive, Neutral]"
8,5d1845236890522b32e94bf9,29-Jun-2019 13:30:27.000000,50,3192272,36,CNBC,How to use this $70 gadget to detect spy camer...,1144961327636582400,1.561840e+09,CNBC,[Neutral]
9,5d1845236890522b32e94c0b,29-Jun-2019 12:31:37.000000,52,3192272,32,CNBC,80% of the stock market is now on autopilot ht...,1144946521919692801,1.561837e+09,CNBC,[Negative]


## Update MongoDB records with an array of sentence sentiments

In [120]:
for i, sentiment in zip(df_clean['tweet_id'], df_clean['sentiment_by_sentence']):
    db['capstone_finance'].update_one({'tweet_id': i}, {'$set': {'sentiment_by_sentence': sentiment}}, upsert = False)

## How to connect to and run CoreNLP server

Source: https://towardsdatascience.com/natural-language-processing-using-stanfords-corenlp-d9e64c1e1024

To start CoreNLP server:
- Open command prompt
- Go to the CoreNLP directory
- Start server by entering the command: java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [122]:
# Examaple code
nlp = StanfordCoreNLP('http://localhost:9000')

text = "This movie was actually neither that funny, nor super witty. The movie was meh. I liked watching that movie. If I had a choice, I would not watch that movie again."
result = nlp.annotate(text,
                   properties={
                       'annotators': 'sentiment, ner, pos',
                       'outputFormat': 'json',
                       'timeout': 1000,
                   })

Exception: Check whether you have started the CoreNLP server e.g.
$ cd stanford-corenlp-full-2015-12-09/ 
$ java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer

In [11]:
for s in result["sentences"]:
    print(s["index"], s["sentiment"])


0 Negative
1 Neutral
2 Positive
3 Negative


## Stanford Demo code

In [7]:

from stanfordcorenlp import StanfordCoreNLP
import logging
import json

class StanfordNLP:
    def __init__(self, host='http://localhost', port=9000):
        self.nlp = StanfordCoreNLP(host, port=port,
                                   timeout=30000)  # , quiet=False, logging_level=logging.DEBUG)
        self.props = {
            'annotators': 'tokenize,ssplit,pos,lemma,ner,parse,depparse,dcoref,relation',
            'pipelineLanguage': 'en',
            'outputFormat': 'json'
        }

    def word_tokenize(self, sentence):
        return self.nlp.word_tokenize(sentence)

    def pos(self, sentence):
        return self.nlp.pos_tag(sentence)

    def ner(self, sentence):
        return self.nlp.ner(sentence)

    def parse(self, sentence):
        return self.nlp.parse(sentence)

    def dependency_parse(self, sentence):
        return self.nlp.dependency_parse(sentence)

    def annotate(self, sentence):
        return json.loads(self.nlp.annotate(sentence, properties=self.props))

    @staticmethod
    def tokens_to_dict(_tokens):
        tokens = defaultdict(dict)
        for token in _tokens:
            tokens[int(token['index'])] = {
                'word': token['word'],
                'lemma': token['lemma'],
                'pos': token['pos'],
                'ner': token['ner']
            }
        return tokens

if __name__ == '__main__':
    sNLP = StanfordNLP()
    text = 'A blog post using Stanford CoreNLP Server. Visit www.khalidalnajjar.com for more details.'
    print( "Annotate:", sNLP.annotate(text))
    print( "POS:", sNLP.pos(text))
    print( "Tokens:", sNLP.word_tokenize(text))
    print( "NER:", sNLP.ner(text))
    print( "Parse:", sNLP.parse(text))
    print( "Dep Parse:", sNLP.dependency_parse(text))

Annotate: {'sentences': [{'index': 0, 'parse': '(ROOT\r\n  (S\r\n    (NP (DT A) (NN blog))\r\n    (VP (NN post)\r\n      (S\r\n        (VP (VBG using)\r\n          (NP (NNP Stanford) (NNP CoreNLP) (NN Server)))))\r\n    (. .)))', 'basicDependencies': [{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 3, 'dependentGloss': 'post'}, {'dep': 'det', 'governor': 3, 'governorGloss': 'post', 'dependent': 1, 'dependentGloss': 'A'}, {'dep': 'compound', 'governor': 3, 'governorGloss': 'post', 'dependent': 2, 'dependentGloss': 'blog'}, {'dep': 'acl', 'governor': 3, 'governorGloss': 'post', 'dependent': 4, 'dependentGloss': 'using'}, {'dep': 'compound', 'governor': 7, 'governorGloss': 'Server', 'dependent': 5, 'dependentGloss': 'Stanford'}, {'dep': 'compound', 'governor': 7, 'governorGloss': 'Server', 'dependent': 6, 'dependentGloss': 'CoreNLP'}, {'dep': 'dobj', 'governor': 4, 'governorGloss': 'using', 'dependent': 7, 'dependentGloss': 'Server'}, {'dep': 'punct', 'governor': 3, '


POS: [('A', 'DT'), ('blog', 'NN'), ('post', 'NN'), ('using', 'VBG'), ('Stanford', 'NNP'), ('CoreNLP', 'NNP'), ('Server', 'NN'), ('.', '.'), ('Visit', 'NN'), ('www.khalidalnajjar.com', 'NN'), ('for', 'IN'), ('more', 'JJR'), ('details', 'NNS'), ('.', '.')]
Tokens: ['A', 'blog', 'post', 'using', 'Stanford', 'CoreNLP', 'Server', '.', 'Visit', 'www.khalidalnajjar.com', 'for', 'more', 'details', '.']
NER: [('A', 'O'), ('blog', 'O'), ('post', 'O'), ('using', 'O'), ('Stanford', 'ORGANIZATION'), ('CoreNLP', 'O'), ('Server', 'O'), ('.', 'O'), ('Visit', 'O'), ('www.khalidalnajjar.com', 'URL'), ('for', 'O'), ('more', 'O'), ('details', 'O'), ('.', 'O')]
Parse: (ROOT
  (S
    (NP (DT A) (NN blog))
    (VP (NN post)
      (S
        (VP (VBG using)
          (NP (NNP Stanford) (NNP CoreNLP) (NN Server)))))
    (. .)))
Dep Parse: [('ROOT', 0, 3), ('det', 3, 1), ('compound', 3, 2), ('acl', 3, 4), ('compound', 7, 5), ('compound', 7, 6), ('dobj', 4, 7), ('punct', 3, 8), ('ROOT', 0, 2), ('compound', 2, 1

In [ ]:
text = "This movie was actually neither that funny, nor super witty. The movie was meh. I liked watching that movie. If I had a choice, I would not watch that movie again."
result = nlp.annotate(text,
                   properties={
                       'annotators': 'sentiment, ner, pos',
                       'outputFormat': 'json',
                       'timeout': 1000,
                   })

In [6]:
from pycorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP('http://localhost:9000')
res = nlp.annotate("I love you. I hate him. You are nice. He is dumb",
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json',
                       'timeout': 1000,
                   })
for s in res["sentences"]:
    print("%d: '%s': %s %s" % (
        s["index"],
        " ".join([t["word"] for t in s["tokens"]]),
        s["sentimentValue"], s["sentiment"]))

Exception: Check whether you have started the CoreNLP server e.g.
$ cd stanford-corenlp-full-2015-12-09/ 
$ java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer